# Notebook 1: Extract Z-scored LFP

### Overview
This Jupyter notebook focuses on extracting local field potential (LFP) traces from Spikegadgets `.rec` files, specifically for neuroscience research related to social competition trials. The notebook includes procedures for preprocessing and synchronizing raw electrophysiology data with corresponding video data and computes various metrics, including Z-scored LFPs.

### Inputs & Data Sources
- **Electrophysiology and LFP Parameters**: Constants like `EPHYS_SAMPLING_RATE`, `LFP_SAMPLING_RATE`, `TRIAL_DURATION`, etc., define basic parameters for LFP data processing.
- **Recording Information**: Stream IDs (`ECU_STREAM_ID`, `TRODES_STREAM_ID`), recording extension (`RECORDING_EXTENSION`), and paths to recording directories (`ALL_SESSION_DIR`).
- **DataFrames for Mapping and Timestamps**: `CHANNEL_MAPPING_DF` for channel mapping, and `TONE_TIMESTAMP_DF` for tone timestamps, loaded from external sources.
- **Constants for DataFrame Columns**: Names for various columns in the DataFrame, defined in an all-caps snake case format, such as `EPHYS_INDEX_COL`, `LFP_INDEX_COL`, etc.

### Output & Utility
- **Processed Data**: The notebook outputs processed data, particularly the Z-scored LFP traces, which are critical for further analysis in neuroscience research.
- **Data Files**: Outputs are saved in various formats (`CSV`, `Pickle`) in a specified output directory (`OUTPUT_DIR`).
- **Visualization**: While not explicitly mentioned, the notebook has the potential for data visualization (plots) based on processed LFP data.

### Processing Workflow
1. **LFP Extraction and Preprocessing**: 
    - Iterates through recording sessions to process `.rec` files.
    - Applies a series of preprocessing steps like bandpass filtering, notch filtering, resampling, and Z-scoring on the LFP data.
    - Exception handling for cases where the recording doesn't contain specified stream IDs.

2. **DataFrame Manipulation and Merging**:
    - Filtering `TONE_TIMESTAMP_DF` for trials with obtained LFP.
    - Addition of trial numbers and merging with `CHANNEL_MAPPING_DF`.
    - Dropping unnecessary columns and restructuring for analysis.

3. **LFP Trace Extraction for Each Trial and Brain Region**: 
    - Linking LFP calculations with trials.
    - Creating new rows for each brain region, extracting baseline, trial, and combined LFP traces.
    - Results in a comprehensive DataFrame that combines trial information with corresponding LFP traces.

4. **Data Storage**:
    - Saving processed DataFrame in both `CSV` and `Pickle` formats for easy access and future use.

### Usage Notes
- The notebook is project-specific and tailored for a particular dataset structure, requiring modifications for different data formats.
- Users should ensure file paths and directory names match their project's structure and adjust constants and parameters as needed for their specific analysis requirements.
- The notebook forms a part of a larger research framework, thus necessitating compatibility checks with other components of the project.

### Dependencies
- Python Libraries: `sys`, `os`, `glob`, `numpy`, `pandas`, `spikeinterface`
- External Data: Channel mapping and tone timestamp files, along with Spikegadgets `.rec` files.

### Customization and Scalability
- The notebook's modular design allows for easy adaptation to different datasets or extensions to include additional processing steps.
- Functions and processing steps are clearly demarcated, facilitating straightforward updates or enhancements.

### Conclusion
This notebook is a vital tool in the preprocessing and analysis of LFP data from Spikegadgets recordings, integral to neuroscience research focused on social competition trials. It offers a structured approach to handle, process, and store electrophysiological data, ensuring reproducibility and efficiency in research workflows.

In [1]:
import sys
import os
import git

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/projects/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
from utilities import helper

## Inputs & Data

Explanation of each input and where it comes from.

In [174]:
EPHYS_SAMPLING_RATE = 20000
LFP_SAMPLING_RATE = 1000
TRIAL_DURATION = 10
FRAME_RATE = 22
ECU_STREAM_ID = "ECU"
TRODES_STREAM_ID = "trodes"
LFP_FREQ_MIN = 0.5
LFP_FREQ_MAX = 300
ELECTRIC_NOISE_FREQ = 60
RECORDING_EXTENTION = "*.rec"

In [175]:
EPHYS_INDEX_COL = "time_stamp_index"
LFP_INDEX_COL = "lfp_index"
EPHYS_TIMESTAMP_COL = "time"

In [176]:
RECORDING_FILE_COL = "recording_file"
BASELINE_LFP_INDEX_RANGE_COL = "baseline_lfp_index_range"
TRIAL_LFP_INDEX_RANGE_COL = "trial_lfp_index_range"
BASELINE_EPHYS_INDEX_RANGE_COL = "baseline_ephys_index_range"
TRIAL_EPHYS_INDEX_RANGE_COL = "trial_ephys_index_range"
BASELINE_VIDEOFRAME_RANGE_COL = "baseline_videoframe_range"
TRIAL_VIDEOFRAME_RANGE_COL = "trial_videoframe_range"
CURRENT_SUBJECT_COL = "current_subject"
ALL_CH_LFP_COL = "all_ch_lfp"
SUBJECT_COL = "Subject"
TRIAL_NUMBER_COL = "trial_number"
SPIKE_INTERFACE_COL = "spike_interface"
EIB_COL = "eib"

In [177]:
SPIKE_INTERFACE_COL.upper()

'SPIKE_INTERFACE'

In [178]:
# NOTE: Change based on individual project data location

# Spreadsheet of channel mapping
CHANNEL_MAPPING_DF = pd.read_excel("../../data/channel_mapping.xlsx")
# Spreadsheet of tone time
TONE_TIMESTAMP_DF = pd.read_pickle("../../proc/rce_tone_timestamps.pkl")

In [179]:
CHANNEL_MAPPING_DF.head()

,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,NaN,15,14,13,31,21.0,15.0,14.0,13.0,16.0
1,1,6.2,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
2,1,6.3,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
3,1,6.4,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
4,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0


In [180]:
TONE_TIMESTAMP_DF.head()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,lfp_index,...,baseline_lfp_index_range,trial_lfp_index_range,baseline_ephys_index_range,trial_ephys_index_range,baseline_videoframe_range,trial_videoframe_range,all_subjects,current_subject,trial_outcome,trial_number
0,6310663,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,1390826,20221202_134600_omission_and_competition_subje...,1734,1.0,6_1_top_2_base_3,rewarded,69541,...,"(59541, 69541)","(69541, 79541)","(1190826, 1390826)","(1390826, 1590826)","(1514, 1734)","(1734, 1954)","(6.1, 6.2)",6.1,rewarded,1
1,7910662,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,2990825,20221202_134600_omission_and_competition_subje...,3728,1.0,6_1_top_2_base_3,rewarded,149541,...,"(139541, 149541)","(149541, 159541)","(2790825, 2990825)","(2990825, 3190825)","(3508, 3728)","(3728, 3948)","(6.1, 6.2)",6.1,rewarded,2
2,9710660,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,4790823,20221202_134600_omission_and_competition_subje...,5972,1.0,6_1_top_2_base_3,rewarded,239541,...,"(229541, 239541)","(239541, 249541)","(4590823, 4790823)","(4790823, 4990823)","(5752, 5972)","(5972, 6192)","(6.1, 6.2)",6.1,rewarded,3
3,11310658,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,6390821,20221202_134600_omission_and_competition_subje...,7966,1.0,6_1_top_2_base_3,omission,319541,...,"(309541, 319541)","(319541, 329541)","(6190821, 6390821)","(6390821, 6590821)","(7746, 7966)","(7966, 8186)","(6.1, 6.2)",6.1,omission,4
4,12810657,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,7890820,20221202_134600_omission_and_competition_subje...,9836,1.0,6_1_top_2_base_3,rewarded,394541,...,"(384541, 394541)","(394541, 404541)","(7690820, 7890820)","(7890820, 8090820)","(9616, 9836)","(9836, 10056)","(6.1, 6.2)",6.1,rewarded,5


In [181]:
# NOTE: Change based on individual project data location
# Where all the recording files are being saved
ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/omission/2023_06_18/*.rec")

In [182]:
ALL_SESSION_DIR

['/scratch/back_up/reward_competition_extention/data/omission/2023_06_18/20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.rec']

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [183]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [211]:
TONE_TIMESTAMPS_CSV = "rce_tone_timestamps.csv"
TONE_TIMESTAMPS_PKL = "rce_tone_timestamps.pkl"
FULL_LFP_TRACES_PKL = "rce_lfp_traces.pkl"

# Functions

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [185]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




# Extracting the LFP

In [186]:
recording_name_to_all_ch_lfp = {}
# Going through all the recording sessions 
for session_dir in ALL_SESSION_DIR:
    # Going through all the recordings in each session
    for recording_path in glob.glob(os.path.join(session_dir, RECORDING_EXTENTION)):
        try:
            recording_basename = os.path.splitext(os.path.basename(recording_path))[0]
            # checking to see if the recording has an ECU component
            # if it doesn't, then the next one be extracted
            current_recording = se.read_spikegadgets(recording_path, stream_id=ECU_STREAM_ID)
            current_recording = se.read_spikegadgets(recording_path, stream_id=TRODES_STREAM_ID)
            print(recording_basename)
            # Preprocessing the LFP
            current_recording = sp.bandpass_filter(current_recording, freq_min=LFP_FREQ_MIN, freq_max=LFP_FREQ_MAX)
            current_recording = sp.notch_filter(current_recording, freq=ELECTRIC_NOISE_FREQ)
            current_recording = sp.resample(current_recording, resample_rate=LFP_SAMPLING_RATE)
            current_recording = sp.zscore(current_recording)
            recording_name_to_all_ch_lfp[recording_basename] = current_recording
        except Exception as error:
            # handle the exception
            print("An exception occurred:", error) # An exception occurred: division by zero
    




20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged


20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged
An exception occurred: stream_id trodes is not in ['ECU']


In [187]:
RECORDING_FILE_COL = "recording_file"
RECORDING_DIR_COL = "recording_dir"
BASELINE_LFP_INDEX_RANGE_COL = "baseline_lfp_index_range"
TRIAL_LFP_INDEX_RANGE_COL = "trial_lfp_index_range"
BASELINE_EPHYS_INDEX_RANGE_COL = "baseline_ephys_index_range"
TRIAL_EPHYS_INDEX_RANGE_COL = "trial_ephys_index_range"
BASELINE_VIDEOFRAME_RANGE_COL = "baseline_videoframe_range"
TRIAL_VIDEOFRAME_RANGE_COL = "trial_videoframe_range"
CURRENT_SUBJECT_COL = "current_subject"
ALL_CH_LFP_COL = "all_ch_lfp"
SUBJECT_COL = "Subject"
TRIAL_NUMBER_COL = "trial_number"
SPIKE_INTERFACE_COL = "spike_interface"
EIB_COL = "eib"

In [188]:
RECORDING_TO_SUBJECT = TONE_TIMESTAMP_DF.drop_duplicates(subset=[RECORDING_FILE_COL, CURRENT_SUBJECT_COL])[[RECORDING_DIR_COL, RECORDING_FILE_COL, CURRENT_SUBJECT_COL]].copy()

In [189]:
RECORDING_TO_SUBJECT = RECORDING_TO_SUBJECT[RECORDING_TO_SUBJECT[RECORDING_FILE_COL].isin(recording_name_to_all_ch_lfp)].reset_index(drop=True)

In [190]:
RECORDING_TO_SUBJECT

,recording_dir,recording_file,current_subject
0,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,1.1
1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,1.4


## Adding the LFP trace information

In [191]:
CHANNEL_MAPPING_DF = CHANNEL_MAPPING_DF.drop(columns=[col for col in CHANNEL_MAPPING_DF.columns if "eib" in col], errors="ignore")

- Adding all the brain region to ch information

In [192]:
CHANNEL_MAPPING_DF[SUBJECT_COL] = CHANNEL_MAPPING_DF[SUBJECT_COL].astype(str)

- Merging the recording and the channel dataframes

In [193]:
RECORDING_TO_SUBJECT = pd.merge(RECORDING_TO_SUBJECT, CHANNEL_MAPPING_DF, left_on=CURRENT_SUBJECT_COL, right_on=SUBJECT_COL, how="left")



In [197]:
ALL_BRAIN_REGION_COL = [col for col in RECORDING_TO_SUBJECT.columns if "spike_interface" in col]

for col in ALL_BRAIN_REGION_COL:
    print(col)
    RECORDING_TO_SUBJECT[col] = RECORDING_TO_SUBJECT[col].astype(int).astype(str)


spike_interface_mPFC
spike_interface_vHPC
spike_interface_BLA
spike_interface_LH
spike_interface_MD


In [195]:
RECORDING_TO_SUBJECT


,recording_dir,recording_file,current_subject,Cohort,Subject,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,2,1.1,5,31,30,29,28
1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,2,1.4,15,31,30,29,28


# Extracting the LFP traces

- Maybe might need to just keep it as an array without putting it into Pandas

In [203]:
for col in ALL_BRAIN_REGION_COL:
    print(col)
    brain_region = col.strip(SPIKE_INTERFACE_COL).strip("_")
    trace_column = "{}_lfp_trace".format(brain_region)
    RECORDING_TO_SUBJECT[trace_column] = RECORDING_TO_SUBJECT.apply(lambda row: recording_name_to_all_ch_lfp[row[RECORDING_FILE_COL]].get_traces(channel_ids=[row[col]]).T[0], axis=1)                                                                                                                                                       
                                                                                                                                                  

spike_interface_mPFC
spike_interface_vHPC
spike_interface_BLA
spike_interface_LH
spike_interface_MD


In [207]:
RECORDING_TO_SUBJECT["BLA_lfp_trace"].iloc[0][:5]

array([0.18319474, 0.4846334 , 0.7660871 , 0.85435367, 0.9609397 ],
      dtype=float32)

In [209]:
ALL_TRACE_COL = [col for col in RECORDING_TO_SUBJECT.columns if "lfp_trace" in col]
for index, row in RECORDING_TO_SUBJECT.iterrows():
    # print(row["BLA_lfp_trace"])
    print(row[RECORDING_FILE_COL])
    break

20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged


In [212]:
RECORDING_TO_SUBJECT.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [214]:
example_df = pd.read_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [216]:
example_df["mPFC_lfp_trace"].iloc[0].shape

(3280151,)

In [217]:
example_df

,recording_dir,recording_file,current_subject,Cohort,Subject,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD,mPFC_lfp_trace,vHPC_lfp_trace,BLA_lfp_trace,LH_lfp_trace,MD_lfp_trace
0,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,2,1.1,5,31,30,29,28,"[-0.3153952, -0.28119573, -0.2697959, -0.33819...","[0.9725201, 1.5246606, 2.241502, 2.4265945, 2....","[0.18319474, 0.4846334, 0.7660871, 0.85435367,...","[0.33724487, 0.4972933, 0.52396804, 0.39059433...","[0.0032272236, -0.066158086, -0.07422615, 0.05..."
1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,2,1.4,15,31,30,29,28,"[0.08225485, 0.06580388, -0.13160776, -0.78964...","[-0.75467384, -1.4102968, -2.273455, -2.297038...","[-0.14317, -0.52495664, -1.2980746, -1.0435501...","[-0.8462542, -1.1688362, -1.3489795, -1.064101...","[-0.4082438, -0.6508234, -0.85198706, -0.83423..."


- Next steps, add the timestamps to the data frame

In [ ]:
# CHANNEL_MAPPING_DF = CHANNEL_MAPPING_DF.rename(columns={col: col.split("_")[-1] for col in CHANNEL_MAPPING_DF.columns})

In [ ]:
# CHANNEL_MAPPING_DF = CHANNEL_MAPPING_DF.rename(columns={col: col.split("_")[-1] for col in CHANNEL_MAPPING_DF.columns})

# OLD CODE BELOW

In [ ]:
raise ValueError("Stop here")

ValueError: Stop here

- Filtering for all trials that we got the LFP for

In [ ]:
TONE_TIMESTAMP_DF = TONE_TIMESTAMP_DF[TONE_TIMESTAMP_DF[RECORDING_FILE_COL].isin(recording_name_to_all_ch_lfp.keys())].reset_index(drop=True)

In [ ]:
TONE_TIMESTAMP_DF.head()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,lfp_index,...,baseline_lfp_index_range,trial_lfp_index_range,baseline_ephys_index_range,trial_ephys_index_range,baseline_videoframe_range,trial_videoframe_range,all_subjects,current_subject,trial_outcome,trial_number
0,6310663,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,1390826,20221202_134600_omission_and_competition_subje...,1734,1.0,6_1_top_2_base_3,rewarded,69541,...,"(59541, 69541)","(69541, 79541)","(1190826, 1390826)","(1390826, 1590826)","(1514, 1734)","(1734, 1954)","(6.1, 6.2)",6.1,rewarded,1
1,7910662,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,2990825,20221202_134600_omission_and_competition_subje...,3728,1.0,6_1_top_2_base_3,rewarded,149541,...,"(139541, 149541)","(149541, 159541)","(2790825, 2990825)","(2990825, 3190825)","(3508, 3728)","(3728, 3948)","(6.1, 6.2)",6.1,rewarded,2
2,9710660,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,4790823,20221202_134600_omission_and_competition_subje...,5972,1.0,6_1_top_2_base_3,rewarded,239541,...,"(229541, 239541)","(239541, 249541)","(4590823, 4790823)","(4790823, 4990823)","(5752, 5972)","(5972, 6192)","(6.1, 6.2)",6.1,rewarded,3
3,11310658,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,6390821,20221202_134600_omission_and_competition_subje...,7966,1.0,6_1_top_2_base_3,omission,319541,...,"(309541, 319541)","(319541, 329541)","(6190821, 6390821)","(6390821, 6590821)","(7746, 7966)","(7966, 8186)","(6.1, 6.2)",6.1,omission,4
4,12810657,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,7890820,20221202_134600_omission_and_competition_subje...,9836,1.0,6_1_top_2_base_3,rewarded,394541,...,"(384541, 394541)","(394541, 404541)","(7690820, 7890820)","(7890820, 8090820)","(9616, 9836)","(9836, 10056)","(6.1, 6.2)",6.1,rewarded,5


- Adding trial numbers based on timestamp ordering for each recording

In [ ]:
TONE_TIMESTAMP_DF = TONE_TIMESTAMP_DF.groupby(RECORDING_FILE_COL).apply(lambda x: helper.compute_sorted_index(x, value_column=EPHYS_TIMESTAMP_COL, index_column=TRIAL_NUMBER_COL)).reset_index(drop=True)

In [ ]:
TONE_TIMESTAMP_DF[TRIAL_NUMBER_COL] = TONE_TIMESTAMP_DF[TRIAL_NUMBER_COL] + 1

## Adding the LFP trace information

In [ ]:
CHANNEL_MAPPING_DF

,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,NaN,15,14,13,31,21.0,15.0,14.0,13.0,16.0
1,1,6.2,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
2,1,6.3,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
3,1,6.4,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
4,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0
5,2,1.2,NaN,31,30,29,28,10.0,31.0,30.0,29.0,28.0
6,2,1.3,NaN,15,14,13,12,9.0,31.0,30.0,29.0,28.0
7,2,1.4,NaN,15,14,13,12,15.0,31.0,30.0,29.0,28.0


- Adding all the brain region to ch information

In [ ]:
CHANNEL_MAPPING_DF[SUBJECT_COL] = CHANNEL_MAPPING_DF[SUBJECT_COL].astype(str)

In [ ]:
TONE_TIMESTAMP_DF[CURRENT_SUBJECT_COL] = TONE_TIMESTAMP_DF[CURRENT_SUBJECT_COL].astype(str)

In [ ]:
CHANNEL_MAP_AND_TONE_TIMESTAMP_DF = TONE_TIMESTAMP_DF.merge(CHANNEL_MAPPING_DF, left_on=CURRENT_SUBJECT_COL, right_on=SUBJECT_COL, how="left")

In [ ]:
CHANNEL_MAP_AND_TONE_TIMESTAMP_DF = CHANNEL_MAP_AND_TONE_TIMESTAMP_DF.drop(columns=[col for col in CHANNEL_MAP_AND_TONE_TIMESTAMP_DF.columns if EIB_COL in col], errors="ignore")

In [ ]:
CHANNEL_MAP_AND_TONE_TIMESTAMP_DF = CHANNEL_MAP_AND_TONE_TIMESTAMP_DF.drop(columns=[SUBJECT_COL], errors="ignore")

In [ ]:
CHANNEL_MAP_AND_TONE_TIMESTAMP_DF.head()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,lfp_index,...,all_subjects,current_subject,trial_outcome,trial_number,Cohort,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,6310663,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,1390826,20221202_134600_omission_and_competition_subje...,1734,1.0,6_1_top_2_base_3,rewarded,69541,...,"(6.1, 6.2)",6.1,rewarded,1,1,21.0,15.0,14.0,13.0,16.0
1,7910662,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,2990825,20221202_134600_omission_and_competition_subje...,3728,1.0,6_1_top_2_base_3,rewarded,149541,...,"(6.1, 6.2)",6.1,rewarded,2,1,21.0,15.0,14.0,13.0,16.0
2,9710660,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,4790823,20221202_134600_omission_and_competition_subje...,5972,1.0,6_1_top_2_base_3,rewarded,239541,...,"(6.1, 6.2)",6.1,rewarded,3,1,21.0,15.0,14.0,13.0,16.0
3,11310658,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,6390821,20221202_134600_omission_and_competition_subje...,7966,1.0,6_1_top_2_base_3,omission,319541,...,"(6.1, 6.2)",6.1,omission,4,1,21.0,15.0,14.0,13.0,16.0
4,12810657,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,7890820,20221202_134600_omission_and_competition_subje...,9836,1.0,6_1_top_2_base_3,rewarded,394541,...,"(6.1, 6.2)",6.1,rewarded,5,1,21.0,15.0,14.0,13.0,16.0


In [ ]:
CHANNEL_MAP_AND_TONE_TIMESTAMP_DF.columns

Index(['time', 'recording_dir', 'recording_file', 'time_stamp_index',
       'video_file', 'video_frame', 'video_number', 'subject_info',
       'competition_closeness', 'lfp_index', 'video_name',
       'baseline_lfp_index_range', 'trial_lfp_index_range',
       'baseline_ephys_index_range', 'trial_ephys_index_range',
       'baseline_videoframe_range', 'trial_videoframe_range', 'all_subjects',
       'current_subject', 'trial_outcome', 'trial_number', 'Cohort',
       'spike_interface_mPFC', 'spike_interface_vHPC', 'spike_interface_BLA',
       'spike_interface_LH', 'spike_interface_MD'],
      dtype='object')

In [ ]:
CHANNEL_MAP_AND_TONE_TIMESTAMP_DF.head()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,lfp_index,...,vHPC_baseline-trial_lfp_trace,BLA_baseline_lfp_trace,BLA_trial_lfp_trace,BLA_baseline-trial_lfp_trace,LH_baseline_lfp_trace,LH_trial_lfp_trace,LH_baseline-trial_lfp_trace,MD_baseline_lfp_trace,MD_trial_lfp_trace,MD_baseline-trial_lfp_trace
0,6310663,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,1390826,20221202_134600_omission_and_competition_subje...,1734,1.0,6_1_top_2_base_3,rewarded,69541,...,"[-0.07045184, -0.0958607, -0.054282565, 0.1512...","[2.0367692, 2.1165679, 2.162167, 2.2685654, 2....","[0.31159526, 0.35909453, 0.18239723, -0.509192...","[2.0367692, 2.1165679, 2.162167, 2.2685654, 2....","[3.0443187, 3.1354659, 3.1810396, 3.1901543, 3...","[0.78082824, 1.1788379, 0.8415931, -0.40104795...","[3.0443187, 3.1354659, 3.1810396, 3.1901543, 3...","[1.3860394, 1.4873364, 1.7566382, 1.8208753, 1...","[-0.9857927, -0.8746131, -0.7362562, -1.413216...","[1.3860394, 1.4873364, 1.7566382, 1.8208753, 1..."
1,7910662,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,2990825,20221202_134600_omission_and_competition_subje...,3728,1.0,6_1_top_2_base_3,rewarded,149541,...,"[0.30606127, 0.22405995, 0.28873706, 0.4100066...","[0.30589536, 0.13679792, -0.064599015, -0.3381...","[0.020899683, -0.05129922, 0.113998264, -0.416...","[0.30589536, 0.13679792, -0.064599015, -0.3381...","[-1.1575702, -1.2699852, -1.3489795, -1.427973...","[0.91754913, 0.4466216, 0.64410734, 0.03342066...","[-1.1575702, -1.2699852, -1.3489795, -1.427973...","[-0.15318082, -0.16800477, 0.019765267, 0.1605...","[1.620752, 1.3415675, 1.4601592, 0.94132084, -...","[-0.15318082, -0.16800477, 0.019765267, 0.1605..."
2,9710660,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,4790823,20221202_134600_omission_and_competition_subje...,5972,1.0,6_1_top_2_base_3,rewarded,239541,...,"[-1.2300198, -1.2323297, -1.3039365, -1.096045...","[-2.0025697, -1.9151709, -1.9436705, -1.552276...","[0.6896895, 1.3983787, 1.7574733, 1.4287783, 1...","[-2.0025697, -1.9151709, -1.9436705, -1.552276...","[-0.20356221, -0.08203254, -0.18837102, -0.142...","[-0.58941895, 0.25521234, 0.66233677, 0.416239...","[-0.20356221, -0.08203254, -0.18837102, -0.142...","[-0.32859758, -0.5361329, -0.97591007, -0.7609...","[0.088943705, 0.8770837, 1.2056813, 0.88696635...","[-0.32859758, -0.5361329, -0.97591007, -0.7609..."
3,11310658,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,6390821,20221202_134600_omission_and_competition_subje...,7966,1.0,6_1_top_2_base_3,omission,319541,...,"[0.1605378, 0.4169363, 0.64561605, 0.6756447, ...","[-1.2729807, -1.0468841, -0.8378873, -0.754288...","[2.6789594, 2.3939636, 2.2096665, 1.97597, 1.8...","[-1.2729807, -1.0468841, -0.8378873, -0.754288...","[-2.4792056, -2.1480372, -1.8259835, -1.677109...","[2.7587237, 2.233108, 1.4796239, 0.93274033, 1...","[-2.4792056, -2.1480372, -1.8259835, -1.677109...","[-2.7720788, -2.5620728, -2.3125362, -2.062999...","[2.0802944, 1.7344022, 1.1192082, 0.4521305, 0...","[-2.7720788, -2.5620728, -2.3125362, -2.062999..."
4,12810657,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,7890820,20221202_134600_omission_and_competition_subje...,9836,1.0,6_1_top_2_base_3,rewarded,394541,...,"[-0.30952612, 0.19056645, 0.4331056, 0.3638087...","[-2.1469674, -2.0690687, -2.0937681, -2.152667...","[-0.18809713, 0.108298354, -0.66878986, -0.944...","[-2.1469674, -2.0690687, -2.0937681, -2.152667...","[-2.1176548, -1.4522797, -1.0360405, -1.108958...","[0.5104247, 1.4613944, 0.5468836, -0.036458906...","[-2.1176548, -1.4522797, -1.0360405, -1.108958...","[-2.018528, -1.7220489, -1.4329818, -1.3736861...","[0.31130296, 1.0006167, 0.25200716, -0.2347125...","[-2

In [ ]:
TONE_TIMESTAMP_DF.to_csv(os.path.join(OUTPUT_DIR, TONE_TIMESTAMPS_CSV))
TONE_TIMESTAMP_DF.to_pickle(os.path.join(OUTPUT_DIR, TONE_TIMESTAMPS_PKL))

In [ ]:
CHANNEL_MAP_AND_TONE_TIMESTAMP_DF.columns

Index(['time', 'recording_dir', 'recording_file', 'time_stamp_index',
       'video_file', 'video_frame', 'video_number', 'subject_info',
       'competition_closeness', 'lfp_index', 'video_name',
       'baseline_lfp_index_range', 'trial_lfp_index_range',
       'baseline_ephys_index_range', 'trial_ephys_index_range',
       'baseline_videoframe_range', 'trial_videoframe_range', 'all_subjects',
       'current_subject', 'trial_outcome', 'trial_number', 'Cohort',
       'spike_interface_mPFC', 'spike_interface_vHPC', 'spike_interface_BLA',
       'spike_interface_LH', 'spike_interface_MD'],
      dtype='object')

- Linking up all LFP calculations with all the trials

In [ ]:
CHANNEL_MAP_AND_TONE_TIMESTAMP_DF[ALL_CH_LFP_COL] = CHANNEL_MAP_AND_TONE_TIMESTAMP_DF[RECORDING_FILE_COL].map(recording_name_to_all_ch_lfp)

- Creating a new row for each brain region

In [ ]:
brain_region_col = [col for col in CHANNEL_MAPPING_DF if SPIKE_INTERFACE_COL in col]

In [ ]:
id_cols = [col for col in CHANNEL_MAP_AND_TONE_TIMESTAMP_DF.columns if col not in brain_region_col]

In [ ]:
brain_region_col

['spike_interface_mPFC',
 'spike_interface_vHPC',
 'spike_interface_BLA',
 'spike_interface_LH',
 'spike_interface_MD']

In [ ]:
for col in brain_region_col:
    CHANNEL_MAP_AND_TONE_TIMESTAMP_DF[col] = CHANNEL_MAP_AND_TONE_TIMESTAMP_DF[col].astype(int).astype(str)

In [ ]:
CHANNEL_MAP_AND_TONE_TIMESTAMP_DF.columns

Index(['time', 'recording_dir', 'recording_file', 'time_stamp_index',
       'video_file', 'video_frame', 'video_number', 'subject_info',
       'competition_closeness', 'lfp_index', 'video_name',
       'baseline_lfp_index_range', 'trial_lfp_index_range',
       'baseline_ephys_index_range', 'trial_ephys_index_range',
       'baseline_videoframe_range', 'trial_videoframe_range', 'all_subjects',
       'current_subject', 'trial_outcome', 'trial_number', 'Cohort',
       'spike_interface_mPFC', 'spike_interface_vHPC', 'spike_interface_BLA',
       'spike_interface_LH', 'spike_interface_MD', 'all_ch_lfp'],
      dtype='object')

- Extracting the traces for each trial and brain region

In [ ]:
CHANNEL_MAP_AND_TONE_TIMESTAMP_DF[BASELINE_LFP_INDEX_RANGE_COL].iloc[0]

(59541, 69541)

In [ ]:
for col in brain_region_col:
    print(col)
    brain_region = col.strip(SPIKE_INTERFACE_COL).strip("_")
    baseline_trace_column = "{}_baseline_lfp_trace".format(brain_region)
    trial_trace_column = "{}_trial_lfp_trace".format(brain_region)
    baselineandtrial_trace_column = "{}_baseline-trial_lfp_trace".format(brain_region)
    
    CHANNEL_MAP_AND_TONE_TIMESTAMP_DF[baseline_trace_column] = CHANNEL_MAP_AND_TONE_TIMESTAMP_DF.apply(lambda row: row[ALL_CH_LFP_COL].get_traces(channel_ids=[row[col]], start_frame=row[BASELINE_LFP_INDEX_RANGE_COL][0], end_frame=row[BASELINE_LFP_INDEX_RANGE_COL][1]).T[0], axis=1)

    CHANNEL_MAP_AND_TONE_TIMESTAMP_DF[trial_trace_column] = CHANNEL_MAP_AND_TONE_TIMESTAMP_DF.apply(lambda row: row[ALL_CH_LFP_COL].get_traces(channel_ids=[row[col]], start_frame=row[TRIAL_LFP_INDEX_RANGE_COL][0], end_frame=row[TRIAL_LFP_INDEX_RANGE_COL][1]).T[0], axis=1)

    CHANNEL_MAP_AND_TONE_TIMESTAMP_DF[baselineandtrial_trace_column] = CHANNEL_MAP_AND_TONE_TIMESTAMP_DF.apply(lambda row: np.concatenate([row[baseline_trace_column], row[trial_trace_column]]), axis=1)                                                                                                              
                                                                                                                                                           
                                                                                                                                                    

spike_interface_mPFC
spike_interface_vHPC
spike_interface_BLA
spike_interface_LH
spike_interface_MD


In [ ]:
CHANNEL_MAP_AND_TONE_TIMESTAMP_DF = CHANNEL_MAP_AND_TONE_TIMESTAMP_DF.drop(columns=[ALL_CH_LFP_COL], errors="ignore")

In [ ]:
CHANNEL_MAP_AND_TONE_TIMESTAMP_DF = CHANNEL_MAP_AND_TONE_TIMESTAMP_DF.drop(columns=[col for col in CHANNEL_MAP_AND_TONE_TIMESTAMP_DF if SPIKE_INTERFACE_COL in col], errors="ignore")

In [ ]:
CHANNEL_MAP_AND_TONE_TIMESTAMP_DF.head()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,lfp_index,...,vHPC_baseline-trial_lfp_trace,BLA_baseline_lfp_trace,BLA_trial_lfp_trace,BLA_baseline-trial_lfp_trace,LH_baseline_lfp_trace,LH_trial_lfp_trace,LH_baseline-trial_lfp_trace,MD_baseline_lfp_trace,MD_trial_lfp_trace,MD_baseline-trial_lfp_trace
0,6310663,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,1390826,20221202_134600_omission_and_competition_subje...,1734,1.0,6_1_top_2_base_3,rewarded,69541,...,"[-0.07045184, -0.0958607, -0.054282565, 0.1512...","[2.0367692, 2.1165679, 2.162167, 2.2685654, 2....","[0.31159526, 0.35909453, 0.18239723, -0.509192...","[2.0367692, 2.1165679, 2.162167, 2.2685654, 2....","[3.0443187, 3.1354659, 3.1810396, 3.1901543, 3...","[0.78082824, 1.1788379, 0.8415931, -0.40104795...","[3.0443187, 3.1354659, 3.1810396, 3.1901543, 3...","[1.3860394, 1.4873364, 1.7566382, 1.8208753, 1...","[-0.9857927, -0.8746131, -0.7362562, -1.413216...","[1.3860394, 1.4873364, 1.7566382, 1.8208753, 1..."
1,7910662,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,2990825,20221202_134600_omission_and_competition_subje...,3728,1.0,6_1_top_2_base_3,rewarded,149541,...,"[0.30606127, 0.22405995, 0.28873706, 0.4100066...","[0.30589536, 0.13679792, -0.064599015, -0.3381...","[0.020899683, -0.05129922, 0.113998264, -0.416...","[0.30589536, 0.13679792, -0.064599015, -0.3381...","[-1.1575702, -1.2699852, -1.3489795, -1.427973...","[0.91754913, 0.4466216, 0.64410734, 0.03342066...","[-1.1575702, -1.2699852, -1.3489795, -1.427973...","[-0.15318082, -0.16800477, 0.019765267, 0.1605...","[1.620752, 1.3415675, 1.4601592, 0.94132084, -...","[-0.15318082, -0.16800477, 0.019765267, 0.1605..."
2,9710660,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,4790823,20221202_134600_omission_and_competition_subje...,5972,1.0,6_1_top_2_base_3,rewarded,239541,...,"[-1.2300198, -1.2323297, -1.3039365, -1.096045...","[-2.0025697, -1.9151709, -1.9436705, -1.552276...","[0.6896895, 1.3983787, 1.7574733, 1.4287783, 1...","[-2.0025697, -1.9151709, -1.9436705, -1.552276...","[-0.20356221, -0.08203254, -0.18837102, -0.142...","[-0.58941895, 0.25521234, 0.66233677, 0.416239...","[-0.20356221, -0.08203254, -0.18837102, -0.142...","[-0.32859758, -0.5361329, -0.97591007, -0.7609...","[0.088943705, 0.8770837, 1.2056813, 0.88696635...","[-0.32859758, -0.5361329, -0.97591007, -0.7609..."
3,11310658,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,6390821,20221202_134600_omission_and_competition_subje...,7966,1.0,6_1_top_2_base_3,omission,319541,...,"[0.1605378, 0.4169363, 0.64561605, 0.6756447, ...","[-1.2729807, -1.0468841, -0.8378873, -0.754288...","[2.6789594, 2.3939636, 2.2096665, 1.97597, 1.8...","[-1.2729807, -1.0468841, -0.8378873, -0.754288...","[-2.4792056, -2.1480372, -1.8259835, -1.677109...","[2.7587237, 2.233108, 1.4796239, 0.93274033, 1...","[-2.4792056, -2.1480372, -1.8259835, -1.677109...","[-2.7720788, -2.5620728, -2.3125362, -2.062999...","[2.0802944, 1.7344022, 1.1192082, 0.4521305, 0...","[-2.7720788, -2.5620728, -2.3125362, -2.062999..."
4,12810657,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,7890820,20221202_134600_omission_and_competition_subje...,9836,1.0,6_1_top_2_base_3,rewarded,394541,...,"[-0.30952612, 0.19056645, 0.4331056, 0.3638087...","[-2.1469674, -2.0690687, -2.0937681, -2.152667...","[-0.18809713, 0.108298354, -0.66878986, -0.944...","[-2.1469674, -2.0690687, -2.0937681, -2.152667...","[-2.1176548, -1.4522797, -1.0360405, -1.108958...","[0.5104247, 1.4613944, 0.5468836, -0.036458906...","[-2.1176548, -1.4522797, -1.0360405, -1.108958...","[-2.018528, -1.7220489, -1.4329818, -1.3736861...","[0.31130296, 1.0006167, 0.25200716, -0.2347125...","[-2

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF.tail()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,lfp_index,...,vHPC_baseline-trial_lfp_trace,BLA_baseline_lfp_trace,BLA_trial_lfp_trace,BLA_baseline-trial_lfp_trace,LH_baseline_lfp_trace,LH_trial_lfp_trace,LH_baseline-trial_lfp_trace,MD_baseline_lfp_trace,MD_trial_lfp_trace,MD_baseline-trial_lfp_trace
578,65281838,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,62088794,20230621_111240_standard_comp_to_omission_D5_s...,61972,2.0,1-4_t3b3L_box1,rewarded,3104439,...,"[-0.54008055, -0.7600229, -0.74047244, -0.6133...","[-0.28906703, -0.27997687, 0.06544914, 0.36360...","[-1.6180482, -1.9034792, -1.7380383, -1.474423...","[-0.28906703, -0.27997687, 0.06544914, 0.36360...","[-0.1579293, -0.14805873, 0.13818814, 0.375082...","[-2.9940765, -3.138845, -2.5860925, -2.3130064...","[-0.1579293, -0.14805873, 0.13818814, 0.375082...","[-0.36269733, -0.47723332, -0.3054293, -0.0349...","[-3.2515497, -3.33427, -2.6852329, -2.262086, ...","[-0.36269733, -0.47723332, -0.3054293, -0.0349..."
579,66781856,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,63588812,20230621_111240_standard_comp_to_omission_D5_s...,63469,2.0,1-4_t3b3L_box1,rewarded,3179440,...,"[-0.82844937, -0.7673543, -0.7062592, -0.71359...","[0.9290142, 0.9490125, 0.92719615, 0.87992734,...","[-1.9343857, -1.6580449, -1.2708042, -0.987191...","[0.9290142, 0.9490125, 0.92719615, 0.87992734,...","[-0.35205075, -0.47049773, -0.44746637, -0.490...","[-2.3623593, -2.257073, -1.7174813, -1.2338227...","[-0.35205075, -0.47049773, -0.44746637, -0.490...","[0.15271465, -0.066812664, -0.32451865, -0.489...","[-2.4020743, -2.2589045, -1.8166682, -1.523965...","[0.15271465, -0.066812664, -0.32451865, -0.489..."
580,68781880,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,65588836,20230621_111240_standard_comp_to_omission_D5_s...,65465,2.0,1-4_t3b3L_box1,rewarded,3279441,...,"[-0.4374408, -0.3470201, -0.22971752, -0.22238...","[-0.4835964, -0.19634742, 0.36360633, 0.823568...","[-0.8435667, -1.4744236, -1.7943972, -1.425336...","[-0.4835964, -0.19634742, 0.36360633, 0.823568...","[1.0199602, 0.78964657, 0.6415878, 0.5922349, ...","[-1.1285365, -1.2864658, -1.2601442, -0.694230...","[1.0199602, 0.78964657, 0.6415878, 0.5922349, ...","[1.008553, 0.66494507, 0.6713082, 0.8367491, 0...","[-0.81765974, -0.9131064, -0.85265684, -0.4231...","[1.008553, 0.66494507, 0.6713082, 0.8367491, 0..."
581,70681909,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,67488865,20230621_111240_standard_comp_to_omission_D5_s...,67362,2.0,1-4_t3b3L_box1,omission,3374443,...,"[0.75024766, 0.7820171, 0.73802865, 1.0141784,...","[1.8689365, 1.7525826, 1.319891, 1.1871747, 1....","[-0.3672424, -0.37633255, -0.18907529, -0.1636...","[1.8689365, 1.7525826, 1.319891, 1.1871747, 1....","[1.4509755, 1.3950422, 1.1351169, 1.6911597, 2...","[0.9146739, 0.8258387, 0.54288197, -0.09212543...","[1.4509755, 1.3950422, 1.1351169, 1.6911597, 2...","[1.6894059, 1.5907778, 1.4094291, 1.8484837, 2...","[0.07635733, 0.09226511, -0.038178664, -0.5472...","[1.6894059, 1.5907778, 1.4094291, 1.8484837, 2..."
582,71681918,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,68488874,20230621_111240_standard_comp_to_omission_D5_s...,68359,2.0,1-4_t3b3L_box1,rewarded,3424443,...,"[0.4276656, 0.8626626, 1.1754694, 0.9017635, 0...","[-0.9671928, -0.8762913, -0.52177507, -0.17634...","[-2.7997687, -2.1725478, -1.2998927, -0.634493...","[-0.9671928, -0.8762913, -0.52177507, -0.17634...","[-0.023031358, 0.6876505, 1.3127874, 1.4049128...","[-0.23689397, 0.19412144, 0.33559978, 0.164509...","[-0.023031358, 0.6876505, 1.3127874, 1.4049128...","[-0.486778, 0.12408066, 0.658582, 0.801752, 0....","[-0.33724487, 0.022270888, 0.2449797

In [ ]:
CHANNEL_MAP_AND_TONE_TIMESTAMP_DF.columns

Index(['time', 'recording_dir', 'recording_file', 'time_stamp_index',
       'video_file', 'video_frame', 'video_number', 'subject_info',
       'competition_closeness', 'lfp_index', 'video_name',
       'baseline_lfp_index_range', 'trial_lfp_index_range',
       'baseline_ephys_index_range', 'trial_ephys_index_range',
       'baseline_videoframe_range', 'trial_videoframe_range', 'all_subjects',
       'current_subject', 'trial_outcome', 'trial_number', 'Cohort',
       'mPFC_baseline_lfp_trace', 'mPFC_trial_lfp_trace',
       'mPFC_baseline-trial_lfp_trace', 'vHPC_baseline_lfp_trace',
       'vHPC_trial_lfp_trace', 'vHPC_baseline-trial_lfp_trace',
       'BLA_baseline_lfp_trace', 'BLA_trial_lfp_trace',
       'BLA_baseline-trial_lfp_trace', 'LH_baseline_lfp_trace',
       'LH_trial_lfp_trace', 'LH_baseline-trial_lfp_trace',
       'MD_baseline_lfp_trace', 'MD_trial_lfp_trace',
       'MD_baseline-trial_lfp_trace'],
      dtype='object')

In [ ]:
CHANNEL_MAP_AND_TONE_TIMESTAMP_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))